In [70]:
import random
from itertools import product


suit_dict = ['spade', 'bastoni', 'denari', 'coppe']
card_dict = {
    'asso': (12, 11),
    'tre': (11, 10),
    're': (10, 4),
    'cavallo': (9, 3),
    'fante': (8, 2),
    'due': (2, 0),
    'quattro': (4, 0),
    'cinque': (5, 0),
    'sei': (6, 0),
    'sette': (7, 0),
}


class Card:

    def __init__(self, name, suit, card_id=None, is_briscola=None):
        """ Initialize a new card. """
        self._name = name
        self._suit = suit
        self._value, self._score = card_dict[name]
        self._card_id = card_id
        self._is_briscola = is_briscola

    @property
    def name(self):
        return self._name

    @property
    def suit(self):
        return self._suit

    @property
    def value(self):
        return self._value

    @property
    def score(self):
        return self._score

    @property
    def card_id(self):
        return self._card_id

    def is_briscola(self):
        return self._is_briscola

    def set_briscola(self):
        self._is_briscola = True

    def __str__(self):
        if self._is_briscola:
            return self._name + ' di ' + self._suit + ' (briscola)'
        else:
            return self._name + ' di ' + self._suit

    def __repr__(self):
        return self.__str__()



class Deck:

    def __init__(self):
        """ Initialize a new deck of cards. """
        self.cards = []
        self.reset()

    def reset(self, deck_state=None):
        """Reinitialize the deck with all cards and shuffle."""
        self.cards = []
        for i, (suit, name) in enumerate(product(suit_dict, card_dict)):
            self.cards.append(Card(name, suit, card_id=i+1))
        if deck_state:
            self.cards = [self.cards[i-1] for i in deck_state]
        else:
            random.shuffle(self.cards)

    def shuffle(self):
        """Shuffle the deck."""
        random.shuffle(self.cards)

    def draw(self):
        """Draw the top card from the deck."""
        if len(self.cards) == 0:
            raise ValueError("No cards left in the deck.")
        return self.cards.pop()

    def get_state(self):
        """Return the current state of the deck."""
        return [card.card_id for card in self.cards]

    def __len__(self):
        """Return the number of cards left in the deck."""
        return len(self.cards)

    def __getitem__(self, position):
        """Return the card at the given position."""
        return self.cards[position]

    def __str__(self):
        return '\n'.join(str(card) for card in self.cards)



# Example usage
deck = Deck()
card_drawn = deck.draw()
print(f"Drawn card: {card_drawn}")
print(f"Cards left in deck: {len(deck)}")
print(deck.get_state())

another_deck = Deck()
print(another_deck.get_state())
another_deck.reset(deck.get_state())
print(another_deck.get_state())

Drawn card: quattro di coppe
Cards left in deck: 39
[26, 2, 20, 36, 31, 34, 15, 18, 8, 23, 14, 38, 40, 16, 35, 11, 22, 7, 3, 5, 17, 28, 30, 29, 1, 4, 33, 39, 25, 19, 6, 32, 12, 21, 27, 9, 13, 10, 24]
[20, 35, 13, 28, 26, 40, 17, 39, 4, 27, 25, 10, 38, 30, 36, 8, 21, 23, 14, 37, 22, 3, 34, 12, 7, 6, 33, 2, 9, 18, 1, 32, 29, 5, 24, 11, 15, 31, 16, 19]
[26, 2, 20, 36, 31, 34, 15, 18, 8, 23, 14, 38, 40, 16, 35, 11, 22, 7, 3, 5, 17, 28, 30, 29, 1, 4, 33, 39, 25, 19, 6, 32, 12, 21, 27, 9, 13, 10, 24]


In [71]:
class Hand(object):
    """docstring for Hand"""

    def __init__(self, hand_state=[]):
        super(Hand, self).__init__()
        self.cards = hand_state

    def add_card(self, card):
        self.cards.append(card)

    def get_card(self, i):
        card_idx = i % len(self.cards)
        return self.cards.pop(card_idx)

    def highest_card(self):
        high_card = self.cards[0]
        for card in self.cards:
            if card.score > high_card.score:
                high_card = card
        return high_card

    def reset(self, hand_state=[]):
        self.cards = hand_state
    
    def get_hand_state(self):
        return self.cards.copy()

    def __getitem__(self, i):
        return self.cards[i]
    
    def __len__(self):
        return len(self.cards)

    def __str__(self):
        out_str = ''
        for card in self.cards:
            out_str += str(card) + ', '
        return out_str

    def __repr__(self):
        return self._str()


class Player(object):
    """docstring for Player"""

    def __init__(self, player_id=0, team_id=0):
        super(Player, self).__init__()
        self.player_id = player_id
        self.team_id = team_id
        self.hand = Hand()

    def reset(self, player_state=None):
        if player_state:
            self.player_id = player_state['player_id']
            self.team_id = player_state['team_id']
            self.hand.reset(player_state['hand'])
        else:
            self.hand.reset()

    def draw(self, deck):
        if len(deck) == 0:
            raise ValueError("No cards left in the deck.")
        self.hand.add_card(deck.draw())

    def hand_size(self):
        return len(self.hand)

    def play(self, hand_card_idx):
        return self.hand.get_card(hand_card_idx)

    def play(self, hand_card_idx, field):
        card_to_play = self.hand.get_card(hand_card_idx)
        field.add_card(card_to_play, self.player_id, self.team_id)
    
    def get_state(self):
        return {
            'player_id': self.player_id,
            'team_id': self.team_id,
            'hand': self.hand.get_hand_state(),
        }

    def play_random(self):
        return self.hand.get_card(random.randint(0, len(self.hand) - 1))

    def play_dummy_ai(self):
        pass


class Field(object):
    """docstring for Field"""

    def __init__(self):
        super(Field, self).__init__()
        self.cards = []
        self.score = 0

    def get_cards(self):
        cards = []
        for card in self.cards:
            card.append(self.cards[0])
        return cards

    def get_cards_and_ids(self):
        return self.cards

    def get_score(self):
        score = 0
        if self.cards != []:
            for card, _, _ in self.cards:
                score += card.score
        return score

    def add_card(self, card, player_id, team_id):
        self.cards.append((card, player_id, team_id))
        self.score += card.score

    def get_current_winner(self):
        # Get the fist card played this round, i.e., from the winning player
        winner_card, winner_player_id, winner_team_id = self.cards[0]
        # Check if any of the following cards played is better than the first one
        for card, player_id, team_id in self.cards[1:]:
            if card.suit == winner_card.suit:
                if card.score > winner_card.score:
                    winner_card = card
                    winner_player_id = player_id
                    winner_team_id = team_id
            elif card.suit != winner_card.suit:
                if card.is_briscola():
                    winner_card = card
                    winner_player_id = player_id
                    winner_team_id = team_id
        return winner_player_id, winner_team_id

    def get_teams_scores(self):
        scores = {}
        for _, _, team_id in self.cards:
            scores[team_id] = 0
        for card, _, team_id in self.cards:
            scores[team_id] += card.value
        return scores

    def get_players_scores(self):
        scores = {}
        for card, player_id, _ in self.cards:
            scores[player_id] = 0
        for card, player_id, _ in self.cards:
            scores[player_id] += card.value
        return scores

    def get_winner_and_score(self):
        winner_player_id, winner_team_id = self.get_current_winner()
        return winner_player_id, winner_team_id, self.score

    def clear_field(self):
        self.score = 0
        self.cards = []

    def reset(self, field_state=None):
        if field_state:
            self.cards = field_state['cards']
            self.score = field_state['score']
        else:
            self.clear_field()
    
    def get_field_state(self):
        return {
            'cards': self.cards.copy(),
            'score': self.score.copy(),
        }

    def __str__(self):
        out_str = ''
        for i, (card, player_id, team_id) in enumerate(self.cards):
            out_str += '{} giocata da giocatore {} in team {}'.format(
                card, player_id, team_id)
            if i != len(self.cards) - 1:
                out_str += '\n'
        return out_str

    def __repr__(self):
        return self._str()

In [72]:
class Game(object):

    def __init__(self, num_players=2, num_players_per_team=2, init_game=True, starting_player=0, starting_team=0):
        """ Initialize the game with a number of players and a number of players per team.

        Args:
            num_players (int): the number of players in the game
            num_players_per_team (int): the number of players per team
            init_game (bool): whether to initialize the game or not
        """
        super(Game, self).__init__()
        assert num_players % 2 == 0, 'The number of players must be an even number.'
        self.num_players = num_players
        self.deck = Deck()
        self.field = Field()
        self.players = []
        self.teams_score = {}
        self.players = [Player(player_id=i, team_id=(i % num_players_per_team)) for i in range(num_players)]
        if num_players == 2:
            num_teams = 2
        else:
            num_teams = num_players // 2
        for i in range(num_teams):
            self.teams_score[i] = 0
        self.last_winner_id = starting_player
        self.last_winner_team_id = starting_team
        self.turn_cnt = 0
        self.played_cards = []  # Track played cards
        # Briscola logic
        self.briscola_card = None
        self.briscola_suit = None
        # Initialize the game
        if init_game:
            self.reset()
            self.game_started = True
        else:
            self.game_started = False

    def init_game(self):
        """Initializes the game by drawing the Briscola card and dealing initial hands to players."""
        self.briscola_card = self.deck.draw()
        self.briscola_suit = self.briscola_card.suit

        # Mark all cards of the briscola suit as Briscola
        for card in self.deck.cards:
            if card.suit == self.briscola_suit:
                card.set_briscola()

        # Deal cards to players
        for _ in range(3):  # Each player draws 3 cards to start
            for i in range(self.num_players):
                # player.hand.add_card(self.deck.draw())
                player_idx = (self.last_winner_id + i) % self.num_players
                self.players[player_idx].draw(self.deck)
        self.game_started = True

    def player_play_card(self, player_id, hand_card_idx) -> None:
        """ Play a card from the player's hand to the field. 
        
        Args:
            player_id (int): the index of the player
            hand_card_idx (int): the index of the card in the player's hand (0-2)
        """
        # Logging
        played_card = self.players[player_id].hand[hand_card_idx % len(self.players[player_id].hand)]
        if played_card:
            self.played_cards.append((player_id, played_card.card_id))
        # Play the card on the field
        self.players[player_id].play(hand_card_idx, self.field)

    def determine_winners(self, verbose=0) -> tuple:
        """ Determine the winner of the turn and update the scores.
        
        Args:
            verbose (int): the verbosity level of the game

        Returns:
            tuple: the index of the winning player and the index of the winning team
        """
        # self.random_step()
        if verbose == 1:
            print(self.field)
        winner_player_id, winner_team_id, score = self.field.get_winner_and_score()
        if verbose:
            print(f'Vince giocatore {winner_player_id} con {score} punti (briscola: {self.briscola_suit})')
        self.last_winner_id = winner_player_id
        self.last_winner_team_id = winner_team_id
        self.teams_score[winner_team_id] += score
        return winner_player_id, winner_team_id, score
    
    def step(self, hand_card_idx, verbose=0):
        """Simulate a single step of the game, i.e., a player plays a card and the turn winner is determined.

        Args:
            hand_card_idx (int): the index of the card to play
            verbose (int): the verbosity level of the game

        Returns:
            int: the index of the winning player
        """
        done = False
        if self.game_started:
            # The current player plays a card
            curr_player_id = (self.last_winner_id + (self.turn_cnt % self.num_players)) % self.num_players

            self.player_play_card(curr_player_id, hand_card_idx)
            self.turn_cnt += 1

            if verbose:
                print(f'Player {curr_player_id} played card {hand_card_idx}')
                print(f'\tfield: {self.field}')
                print(f'\tplayer hand: {self.players[curr_player_id].hand}')

            # If all players have played a card, prepare for next turn
            if self.turn_cnt % self.num_players == 0:
                # Determine the winner of the turn
                winner_player_id, winner_team_id, score = self.determine_winners(verbose)
                self.last_winner_id = winner_player_id
                self.last_winner_team_id = winner_team_id
                # Clear the field
                self.field.clear_field()
                # Each player draws a card from the deck
                if len(self.deck) > 0:
                    for i in range(self.num_players):
                        curr_player = self.players[(winner_player_id + i) % self.num_players]
                        if len(self.deck) > 0:
                            curr_player.draw(self.deck)
                        else:
                            if verbose:
                                print('Il mazzo è finito, il giocatore pesca la briscola.')
                            curr_player.hand.add_card(self.briscola_card)
                elif verbose:
                    print('Il mazzo è finito.')
                # Check if the game is over by checking if the players have no more cards
                if self.players_hand_size() == 0:
                    self.game_started = False
                    done = True
                if verbose:
                    print('-' * 80)
                return score, done
            else:
                return 0, done
        else:
            print('Il gioco non è ancora iniziato.')
            return None
    
    def reset(self, game_state=None):
        if game_state:
            self.players = [Player(player_state=player_state) for player_state in game_state['players']]
            self.field.reset(game_state['field'])
            self.deck.reset(game_state['deck'])
            self.briscola_card = game_state['briscola']
            self.last_winner_id = game_state['last_winner_id']
            self.last_winner_team_id = game_state['last_winner_team_id']
            self.turn_cnt = game_state['turn_cnt']
            self.played_cards = game_state['played_cards']
            self.game_started = game_state['game_started']
            # Set the briscola suit
            self.briscola_suit = self.briscola_card.suit
            # Set the briscola flag for the cards in deck
            for card in self.deck.cards:
                if card.suit == self.briscola_suit:
                    card.set_briscola()
        else:
            for player in self.players:
                player.reset()
            self.turn_cnt = 0
            self.field.reset()
            self.deck.reset()
            num_teams = 2 if self.num_players == 2 else self.num_players // 2
            self.teams_score = {i: 0 for i in range(num_teams)}
            self.init_game()
    
    def get_game_state(self):
        game_state = {
            'players': [player.get_state() for player in self.players],
            'field': self.field.get_field_state(),
            'deck': self.deck.get_state(),
            'briscola': self.briscola_card,
            'last_winner_id': self.last_winner_id,
            'last_winner_team_id': self.last_winner_team_id,
            'turn_cnt': self.turn_cnt,
            'played_cards': self.played_cards,
            'game_started': self.game_started
        }
        return game_state

    def get_winner_team(self):
        winning_score = 0
        winning_team = 0
        for team_id, score in self.teams_score.items():
            if score > winning_score:
                winning_score = score
                winning_team = team_id
        return winning_team, winning_score

    def players_hand_size(self):
        return sum(player.hand_size() for player in self.players)

    # def random_step(self, debug=False):
    #     for i, player in enumerate(self.players, start=self.last_winner_id):
    #         if not debug:
    #             self.field.add_card(player.play_random(), player.player_id, player.team_id)
    #         else:
    #             if i == self.last_winner_id:
    #                 print('Gioca per primo il giocatore {} team {}'.format(i, player.team_id))
    #             else:
    #                 print('Gioca il giocatore {} team {}'.format(i, player.team_id))
    #             if i == 0:
    #                 hand = player.hand
    #                 index = int(input('Scegli una carta:\n0: {}\n1: {}\n2: {}\n'.format(hand[0], hand[1], hand[2])))
    #                 player.play(index, self.field)
    #             else:
    #                 self.field.add_card(player.play_random(), player.player_id, player.team_id)


    def simulate_random_game(self):
        done = False
        while not done:
            score, done = self.step(random.randint(0, 2), verbose=0)
        winning_team, winning_score = self.get_winner_team()
        print(f'Players hand size: {self.players_hand_size()}')
        print('Vince il team {} con {} punti.'.format(
            winning_team, winning_score))
    
    def get_game_state(self):
        game_state = {
            'players': [player.hand.get_state() for player in self.players],
            'field': self.field.get_cards_and_ids(),
            'deck': self.deck.get_state(),
            'briscola': self.briscola_card.card_id,
            'last_winner_id': self.last_winner_id,
            'last_winner_team_id': self.last_winner_team_id,
            'turn_cnt': self.turn_cnt,
            'played_cards': self.played_cards,
            'game_started': self.game_started
        }
        return game_state

Game().simulate_random_game()

Players hand size: 0
Vince il team 0 con 63 punti.
